In [1]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

In [2]:
# from transformers import pipeline
# generator = pipeline('text-generation', model = 'EleutherAI/polyglot-ko-5.8b', device="cuda")
# generator("안녕하세요, ", max_length = 30, num_return_sequences=1)
# ## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},
# ##  {'generated_text': "Hello, I'm a language modeler. I write and maintain software in Python. I love to code, and that includes coding things that require writing"}, ...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '안녕하세요, ^^*오늘은 제가 좋아하는 맛집을 소개해드리려고 해요.바로바로~~~~~~'}]

In [ ]:
# generator("사람에게 물이 부족하면", max_length = 120, num_return_sequences=1)

In [1]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from datasets import load_dataset
import evaluate

import numpy as np

In [2]:
# str_model = 'KETI-AIR/ke-t5-base'
str_model = "paust/pko-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(str_model)
tokenizer = AutoTokenizer.from_pretrained(str_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
path_dataset = "daekeun-ml/naver-news-summarization-ko"

dataset_paper = load_dataset(path_dataset)
dataset_paper

Found cached dataset csv (/opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [4]:
prefix = "기사를 요약 하시오: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
dataset_paper_tokenized = dataset_paper.map(preprocess_function, batched=True)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-628b665501d0a050.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-314b3fce3cddf558/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-62eb96bed186cb0f.arrow


Map:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [6]:
idx = 109

print("전문")
print(dataset_paper_tokenized["train"][idx]["document"])
print()
print("요약")
print(dataset_paper_tokenized["train"][idx]["summary"])
print()
print(tokenizer.decode(dataset_paper_tokenized["train"][idx]["input_ids"]))
print()
print(tokenizer.decode(dataset_paper_tokenized["train"][idx]["labels"]))

전문
올 1 5월 현대차 FCEV 판매 4 396대 도요타·혼다 판매량 합친 것보다 2배 이상 반도체·우크라이나 사태로 FCEV 시장 축소 현대자동차 수소전기차 넥쏘 . 현대차 제공 현대자동차가 글로벌 수소전기차 FCEV 시장에서 일본 자동차 기업 도요타와 혼다를 합친 것보다 더 많은 판매량을 기록 했다. 2018년 넥쏘 출시 이후 글로벌 강자로 올라선 이후 압도적 우위를 지키고 있다. 5일 에너지전문 시장조사업체 SNE리서치 에 따르면 올 들어 5월까지 FCEV 글로벌 시장 규모는 7 809대로 전년 동기 대비 3.8% 성장했다. 지난해 두 배 가까이 커졌던 글로벌 FCEV 시장은 올해 성장세가 둔화되는 모습이다. 차량용 반도체 수급난 원자재 가격 상승 러시아의 우크라이나 침공 등으로 차량 생산과 공급이 원활하지 못한 것이 주요 원인으로 지목됐다. SNE리서치 2022년 1 5월 글로벌 수소전기차 판매 대수. SNE리서치 제공 올해 1 5월 주요 제조사별 판매 대수는 현대차가 4 396대로 1위를 차지했다. 2위를 기록한 도요타는 1 929대의 FCEV를 판매 지난해 같은 기간보다 40.5% 부진했다. 혼다는 207대를 판매해 3위를 기록했다. 현대차는 도요타와 혼다의 FCEV 판매량을 합친 것 2 136대 보다 2배 이상 많이 판매한 것이다. 시장점유율도 56.3%로 압도적이었다. SNE리서치는 현대차의 넥쏘는 꾸준한 판매실적을 보인 반면 도요타 미라이는 일본에서 겪었던 공급망 이슈 자연재해 등이 겹치면서 판매량이 대폭 줄었다 며 글로벌 FCEV 시장이 우크라이나 사태 반도체 수급난 등으로 어려운 상황에서도 현대차가 시장 성장을 이끌었다 고 평가했다.

요약
S SNE리서치는 올 1 5월 주요 제조사별 판매 대수는 현대차가 4 396대로 1위를 차지했으며 글로벌 수소전기차 FCEV 시장에서 일본 자동차 기업 도요타와 혼다를 합친 것보다 더 많은 판매량을 기록했으며 2018년 넥쏘 출시 이후 압도적 우위를 지키고 있다고 밝혔다.

기사를 요약 하시오: 올 1

In [7]:
rouge = evaluate.load("rouge")

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions, labels)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=str_model)

training_args = Seq2SeqTrainingArguments(
    output_dir="summary_with_news",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=2,
    # eval_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    eval_steps=2000,
    fp16=True,
    resume_from_checkpoint=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_paper_tokenized["train"],
    eval_dataset=dataset_paper_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2000,0.375600,0.322672,0.277900,0.074800,0.274100,0.273500,19.000000
4000,0.334000,0.304106,0.280100,0.079400,0.276700,0.276200,19.000000
6000,0.312800,0.296904,0.280600,0.078800,0.277800,0.277100,19.000000
8000,0.307200,0.289998,0.285500,0.081900,0.282400,0.281700,19.000000
10000,0.294900,0.288371,0.282300,0.079800,0.279400,0.278800,19.000000


[[    0 23531 17357 ...   222 19594   222]
 [    0   831   614 ...   222  9907   222]
 [    0  2534   349 ...   222  2845   274]
 ...
 [    0    44    40 ...   222  1729   719]
 [    0    22   349 ...  1909   330  5512]
 [    0    22   349 ...  2342   311   222]] [[  635   349    13 ...  -100  -100  -100]
 [  831   614   222 ...  -100  -100  -100]
 [  329   222 29877 ...  -100  -100  -100]
 ...
 [   44    40  1918 ...  -100  -100  -100]
 [  885  1272   222 ...  -100  -100  -100]
 [ 7765   589   676 ...  -100  -100  -100]]
[[    0 23531 17357 ...   222 19594   222]
 [    0   831   614 ...   222  9907   222]
 [    0  2534   349 ...   222  2845   274]
 ...
 [    0    44    40 ...   222  1729   719]
 [    0    22   349 ...  1909   330  5512]
 [    0    22   349 ...  2342   311   222]] [[  635   349    13 ...  -100  -100  -100]
 [  831   614   222 ...  -100  -100  -100]
 [  329   222 29877 ...  -100  -100  -100]
 ...
 [   44    40  1918 ...  -100  -100  -100]
 [  885  1272   222 ...  -100  

TrainOutput(global_step=11100, training_loss=0.34318919104498785, metrics={'train_runtime': 8434.7954, 'train_samples_per_second': 10.525, 'train_steps_per_second': 1.316, 'total_flos': 1.2572109356544e+17, 'train_loss': 0.34318919104498785, 'epoch': 4.0})

In [16]:
from transformers import pipeline

summarizer = pipeline("summarization", model="./summary_with_news/checkpoint-11000")
# summarizer = pipeline("text2text-generation", model="./summary/checkpoint-128000")
summarizer("요약: " + "GPT-3를 필두로 하여 거대 파라미터의 모델에 대규모 코퍼스를 학습한 (초)거대 언어모델은 자연스러운 문장을 생성하거나 다양한 태스크를 적은 학습 데이터로 수행하는 등 뛰어난 가능성을 보였다. 하지만 학습에 막대한 자본이 필요한 거대 언어모델은 AI의 독점화 및 권력화, 그리고 데이터 및 컴퓨팅 파워 격차에 따른 기술 격차 심화를 낳을 것이라는 우려도 존재한다. 빅테크 기업이 아닌 일반 기업이나 연구자가 거대 언어 모델을 다루기는 쉽지 않은 것은 엄연한 사실이다.")


[{'summary_text': '빅테크 기업이 아닌 일반 기업이나 연구자가 거대 언어'}]

In [17]:
full_news = """
뚜렷한 이유는 안밝혀..지난해 사업협력 단순투자로 공시했는데네이버 카페 주주모임, 불매운동하자는 글까지3명 사외이사 재선임 최종 부결돼도상법상 임시주총까지 퇴임이사로 활동 가능[이데일리 김현아 기자]KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 현대차그룹은 강충구 고려대 전기전자공학부 교수, 여은정 중앙대 경영학부 교수, 표현명  한국타이어테크놀로지 사외이사 등 3명에 대해 반대의 뜻을 KT에 전했다. 앞서 현대차그룹은 윤경림 차기 CEO 후보에 대해서도 “이사회가 대주주 의견을 고려해야 한다”는 취지를 밝혀, 사실상 최대 주주인 국민연금과 뜻을 함께 하겠다는 의지를 밝힌 바 있다. 지난해 KT와 현대차는 7500억 규모의 자사주 맞교환을 통해 도심항공모빌리티(UAM) 등 미래 모빌리티 분야의 경쟁력을 키우기로 합의한 바 있다. 현대차 그룹이 KT 지분 4.6%, 현대모비스가 KT 지분 3.1%를, KT가 현대차 지분 1.04%와 현대모비스 지분 1.46%를 갖게 됐다. 당시 양사는 “사업협력을 위한 단순투자”라면서 지분투자 목적을 ‘단순투자’로 공시했다. 하지만, 이 같은 보도가 나오자 개인주주들은 반발하고 있다. 네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 당부드린다. KT CFO는 현대차그룹 지분을 매각하시길 바란다”고 했다. 앞서 주주모임 카페 개설자도 이데일리와의 통화에서 “이번 주총에서 현대차, 신한은행이 반대하면, 주주제안으로 미래사업 제휴에서 두 회사를 제외하고 KT와 상호주식교환 등을 한 걸 해지하라고 요구할 예정이다. 카페 안에선 굉장히 좋은 현대차에 대해 불매 운동까지 하자는 글도 있다”고 밝힌 바 있다. 한편 이강철 이사에 이어 어제(28일),김대유 이사(DB생명 사외이사)와 유희열 이사(한국 이산화탄소 포집 및 처리 연구개발센터(KCRC) 이사장) 등 지난 정부 출신 사외이사들이 사퇴하면서 KT 이사회는 사내이사 0명, 사외이사 4명이 남은 상태다. 이중 강충구, 여은정, 표현명 이사가 주총에서 재선임되지 않으면 이사는 1명만 남게 된다. 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다. 퇴임이사 자격으로 가능하다. ESG 자문기관인 서스틴베스트 자문 변호사에 따르면, 상법 386조 1항에 따라, 법률 또는 정관에서 정한 이사의 원수를 결한 경우에는 임기의 만료 또는 사임으로 인하여 퇴임한 이사는 새로 선임된 이사가 취임할 때까지 이사로서의 권리와 의무가 있다고 돼 있다. KT 이사회의 최소 의결 정족 원수는 3명인데, 현대차 주장대로 선임안이 부결되면 이사는 1명 남게 된다. 하지만, 부결돼도 해당 이사들은 이 조항에 근거해 임시주총에서 새로운 이사가 추천돼 선임될 때까지 퇴임이사로 활동할 수 있다. 한편 KT의 주가는 다시 3만 원대 아래로 추락했다. 29일 현재 2만 9200원이 돼 전날보다 2.83% 하락한 채 마감했다.
"""

def get_summary(full_news):
    news_tokenized = len(tokenizer(full_news)["input_ids"])
    print(news_tokenized)
    
    return summarizer(full_news, min_length=news_tokenized // 2, max_length=news_tokenized // 2 + (news_tokenized // 4))

get_summary(full_news)
# 1934

# [{'generated_text': '제주항공에 자금을 지원하기 위해 제주항공 주식을 담보로 돈을 빌리는 AK홀딩스, SK케미칼 주식 공개매수 나서는 SK디스커버리, 하나금융14호스팩으로 본 스팩의 상장폐지 절차와 관련한 이야기 등을 담아봤어요.AK홀딩스 제주항공 유증 참여 위해 교환사채 발행애경그룹 지주회사AK홀딩스가 자회사인 저비용항공사제주항공의 주주배정 유상증자 참여를 위해 1300억원 )                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 '}]


1209


[{'summary_text': 'KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있으며, 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [18]:
list_news = ["뚜렷한 이유는 안밝혀..지난해 사업협력 단순투자로 공시했는데네이버 카페 주주모임, 불매운동하자는 글까지3명 사외이사 재선임 최종 부결돼도상법상 임시주총까지 퇴임이사로 활동 가능[이데일리 김현아 기자]KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 현대차그룹은 강충구 고려대 전기전자공학부 교수, 여은정 중앙대 경영학부 교수, 표현명  한국타이어테크놀로지 사외이사 등 3명에 대해 반대의 뜻을 KT에 전했다. 앞서 현대차그룹은 윤경림 차기 CEO 후보에 대해서도 “이사회가 대주주 의견을 고려해야 한다”는 취지를 밝혀, 사실상 최대 주주인 국민연금과 뜻을 함께 하겠다는 의지를 밝힌 바 있다. 지난해 KT와 현대차는 7500억 규모의 자사주 맞교환을 통해 도심항공모빌리티(UAM) 등 미래 모빌리티 분야의 경쟁력을 키우기로 합의한 바 있다. 현대차 그룹이 KT 지분 4.6%, 현대모비스가 KT 지분 3.1%를, KT가 현대차 지분 1.04%와 현대모비스 지분 1.46%를 갖게 됐다."
,
"당시 양사는 “사업협력을 위한 단순투자”라면서 지분투자 목적을 ‘단순투자’로 공시했다. 하지만, 이 같은 보도가 나오자 개인주주들은 반발하고 있다. 네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 당부드린다. KT CFO는 현대차그룹 지분을 매각하시길 바란다”고 했다. 앞서 주주모임 카페 개설자도 이데일리와의 통화에서 “이번 주총에서 현대차, 신한은행이 반대하면, 주주제안으로 미래사업 제휴에서 두 회사를 제외하고 KT와 상호주식교환 등을 한 걸 해지하라고 요구할 예정이다."
,
"카페 안에선 굉장히 좋은 현대차에 대해 불매 운동까지 하자는 글도 있다”고 밝힌 바 있다. 한편 이강철 이사에 이어 어제(28일),김대유 이사(DB생명 사외이사)와 유희열 이사(한국 이산화탄소 포집 및 처리 연구개발센터(KCRC) 이사장) 등 지난 정부 출신 사외이사들이 사퇴하면서 KT 이사회는 사내이사 0명, 사외이사 4명이 남은 상태다. 이중 강충구, 여은정, 표현명 이사가 주총에서 재선임되지 않으면 이사는 1명만 남게 된다. 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다. 퇴임이사 자격으로 가능하다."
,
"ESG 자문기관인 서스틴베스트 자문 변호사에 따르면, 상법 386조 1항에 따라, 법률 또는 정관에서 정한 이사의 원수를 결한 경우에는 임기의 만료 또는 사임으로 인하여 퇴임한 이사는 새로 선임된 이사가 취임할 때까지 이사로서의 권리와 의무가 있다고 돼 있다. KT 이사회의 최소 의결 정족 원수는 3명인데, 현대차 주장대로 선임안이 부결되면 이사는 1명 남게 된다. 하지만, 부결돼도 해당 이사들은 이 조항에 근거해 임시주총에서 새로운 이사가 추천돼 선임될 때까지 퇴임이사로 활동할 수 있다. 한편 KT의 주가는 다시 3만 원대 아래로 추락했다. 29일 현재 2만 9200원이 돼 전날보다 2.83% 하락한 채 마감했다."]

for news in list_news:
    print(get_summary(news))

436
[{'summary_text': 'KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 앞서 현대차그룹은 윤경림 차기 CEO 후보에 대해서도 “이사회가 대주주 의견을 고려해야 한다”는 취지를 밝혀, 사실상 최대 주주인 국민연금과 뜻을 함께 하겠다는 의지를 밝힌 바 있다. 지난해 KT와 현대차는 7500억 규모의 자사주 맞교환을 통해 도심항공모빌리티(UAM) 등 미래 모빌리티 분야의 경쟁력을 키우기로 합의한 바 있다.'}]
254
[{'summary_text': '네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 당부드린다. KT CFO는 현대차그룹 지분을 매각하시길 바란다”고 했으며, 네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 '}]
265
[{'summary_text': '지난 정부 출신 사외이사들이 사퇴하면서 KT 이사회는 사내이사 0명, 사외이사 4명이 남은 상태이며 이중 강충구, 여은정, 표현명 이사가 주총에서 재선임되지 않으면 이사는 1명만 남게 되며 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다.'}]
252
[{'summary_text': 'KT 이사회의 최소 의결 정족 원수는 3명인데, 현대차 주장대로 선임안이 부결되면 이사는 1명 남게 되고, 부결돼도 해당 이사들은 이 조항에 근거해 임시주총에서 새로운 